In [1]:
import numpy as np
import pandas as pd
import math
import time
import os
import sys
import warnings
warnings.simplefilter(action='ignore')
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import fmin,tpe,hp,STATUS_OK, Trials
import pickle
start = time.time()

In [35]:
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/extract_data.csv"
col_list=['LogP','Exp_RT']
lc_df = pd.read_csv(data_path,usecols=col_list)
col_smile_list=['SMILES','LogP','Exp_RT']
lc_smiles_df = pd.read_csv(data_path,usecols=col_smile_list)

# Remove non_retained molecules
# index=lc_df[lc_df['Exp_RT'] < 180].index
# lc_df.drop(lc_df[lc_df['Exp_RT'] < 180].index,inplace=True)

# Import descriptor file
path = os.getcwd()
dirname = os.path.dirname(path)
data_path = dirname+"/LogP/data/descriptor_1.csv"
des_df = pd.read_csv(data_path,index_col=0)

# Remove non_retained molecules
#des_df  = des_df.drop(des_df.index[[index]])
#lc_smiles_df = lc_smiles_df.drop(lc_smiles_df[[index]])
# combine two dataframe
data_comb = pd.concat([lc_smiles_df,des_df],axis=1)

In [36]:
data_comb

,SMILES,LogP,Exp_RT,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CCSC(=N)N,0.67,93.5,6.635046,0.206019,6.635046,0.206019,0.378489,104.178,96.114,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...,2.64,687.8,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Oc1cccc2c(O)nccc12,1.73,583.6,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NCCc1ccc(S(=O)(=O)F)cc1,1.05,653.1,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,1.33,705.6,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,OCCn1cc(-c2ccc3c(c2)CC/C3=N\O)c(-c2ccncc2)n1,2.05,603.4,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,CC(C)OC(=O)N1CCC(Oc2ncnc3c2cnn3-c2ccc(S(C)(=O)...,1.59,1061.4,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,CCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(=...,3.73,1059.3,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,O=C(c1cc2ccccc2[nH]1)N1CCCC1,1.70,794.3,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
des_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
1,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,413.127297,148.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,161.047678,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,203.041628,72.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,225.090212,84.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.698502,0.432778,5.698502,0.432778,0.616936,133.194,122.106,133.089149,52.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,334.142976,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,477.148218,176.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,402.240624,160.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,214.110613,82.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
lc_smiles_df

,SMILES,LogP,Exp_RT
0,CCSC(=N)N,0.67,93.5
1,COC(=O)N1CCN(C(=O)Cc2ccc(Cl)c(Cl)c2)[C@H](CN2C...,2.64,687.8
2,Oc1cccc2c(O)nccc12,1.73,583.6
3,NCCc1ccc(S(=O)(=O)F)cc1,1.05,653.1
4,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,1.33,705.6
...,...,...,...
2066,OCCn1cc(-c2ccc3c(c2)CC/C3=N\O)c(-c2ccncc2)n1,2.05,603.4
2067,CC(C)OC(=O)N1CCC(Oc2ncnc3c2cnn3-c2ccc(S(C)(=O)...,1.59,1061.4
2068,CCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(=...,3.73,1059.3
2069,O=C(c1cc2ccccc2[nH]1)N1CCCC1,1.70,794.3


In [25]:
des_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
1,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,413.127297,148.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,161.047678,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,203.041628,72.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,225.090212,84.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.698502,0.432778,5.698502,0.432778,0.616936,133.194,122.106,133.089149,52.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,334.142976,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,477.148218,176.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,402.240624,160.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,214.110613,82.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
des_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
1,12.990628,-0.335181,12.990628,0.035905,0.760602,414.333,389.133,413.127297,148.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.356759,-0.052407,9.356759,0.052407,0.617197,161.160,154.104,161.047678,60.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.386283,-4.565776,12.386283,0.311786,0.740582,203.238,193.158,203.041628,72.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.531255,-0.169277,11.531255,0.169277,0.665746,225.251,214.163,225.090212,84.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.698502,0.432778,5.698502,0.432778,0.616936,133.194,122.106,133.089149,52.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,9.236427,0.039104,9.236427,0.039104,0.567510,334.379,316.235,334.142976,126.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2067,14.683496,-3.539697,14.683496,0.054651,0.550748,477.518,453.326,477.148218,176.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,12.960935,-1.190444,12.960935,0.073898,0.724147,402.531,368.259,402.240624,160.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,12.111258,0.133056,12.111258,0.133056,0.777073,214.268,200.156,214.110613,82.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
des_without_lc = pd.concat([des_df,lc_df['LogP']],axis=1)
des_without_lc_feat_corr = des_without_lc.columns[des_without_lc.corrwith(des_without_lc['LogP']) >=0.90][:-1]
des_without_lc = des_without_lc.drop(columns=des_without_lc_feat_corr)

# Filling the nan with mean values in des_with_lc
for col in des_without_lc:
    des_without_lc[col].fillna(des_without_lc[col].mean(),inplace=True)

# Remove columns with zero vlues
des_without_lc = des_without_lc.loc[:,(des_without_lc**2).sum() != 0]
data = des_without_lc.drop(['LogP'],axis=1)

# Remove features with low Variance(threshold<=0.05)
data_var = data.var()
del_feat = list(data_var[data_var <= 0.05].index)
data.drop(columns=del_feat, inplace=True)

# Remove features with correlation(threshold >0.95)
corr_matrix = data.corr().abs()
mask = np.triu(np.ones_like(corr_matrix,dtype=bool))
tri_df = corr_matrix.mask(mask)
to_drop =  [ c for c in tri_df.columns if any(tri_df[c] > 0.95)]
data = data.drop(to_drop,axis=1)


In [11]:
data

,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,...,fr_nitro,fr_para_hydroxylation,fr_phenol_noOrthoHbond,fr_piperdine,fr_piperzine,fr_pyridine,fr_sulfide,fr_sulfonamd,fr_thiazole,fr_unbrch_alkane
1,-0.335181,12.990628,0.035905,1.851852,2.518519,35.498262,6.415298,-0.135196,1.700079,694.777490,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.052407,9.356759,0.052407,1.833333,2.583333,16.266691,5.905835,0.458301,2.943373,384.548091,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-4.565776,12.386283,0.311786,1.769231,2.153846,32.253567,7.859714,0.551821,2.943371,371.635430,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.169277,11.531255,0.169277,2.058824,2.882353,16.149410,6.085148,0.095815,2.333773,714.834641,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.432778,5.698502,0.432778,1.800000,2.300000,14.737623,5.264514,0.990514,2.203815,217.429692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,0.039104,9.236427,0.039104,1.880000,2.640000,16.434137,6.043003,0.269341,1.824794,931.852081,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2067,-3.539697,14.683496,0.054651,1.939394,2.575758,32.227747,7.903055,0.051066,1.526280,1285.437418,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2068,-1.190444,12.960935,0.073898,1.827586,2.517241,16.560861,5.922527,-0.190714,1.694820,769.232708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2069,0.133056,12.111258,0.133056,1.937500,2.687500,16.153602,5.976063,0.078777,1.956469,490.467045,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
